# Napari image viewer

Uses the octopuslite dask image loader to automatically sort and load images from multichannel time lapse experiments. Can also use btrack to load associated tracks and segmentation images.

In [1]:
import napari
from octopuslite import DaskOctopusLiteLoader

In [6]:
import numpy

# testing transform implementation

In [79]:
import os

expt = 'ND0012'
pos = 'Pos0'
image_path = f'/home/nathan/data/kraken/ras/{expt}/{pos}/{pos}_images'
transform_path = f'/home/nathan/data/kraken/ras/{expt}/{pos}/gfp_transform_tensor.npy'
if os.path.exists(image_path):
    print(expt, pos)
#     trans = numpy.load(transform_path)
#     if len(trans.shape) == 4:
#         print('reshape')
#         numpy.save(transform_path, numpy.reshape(trans, (len(trans)*len(trans[0]), 3, 3)))
    images = DaskOctopusLiteLoader(image_path, remove_background=True, 
                                   transforms=transform_path,
                                   crop = (1200, 1600))
    gfp = images['gfp']
    rfp = images['rfp']
    irfp = images['irfp']
    masks = images['mask'].astype(int)
    viewer = napari.Viewer()
    viewer.add_image(gfp,
                     contrast_limits=[0,255],
                     blending = 'additive', 
                     colormap = 'green',
                    name = f'transform_{14}')#, is_pyramid=False)
    viewer.add_image(rfp, name="rfp", 
                     contrast_limits=[0,55], 
                     blending = 'additive', colormap = 'red')#, is_pyramid=False)
    viewer.add_image(irfp, name="irfp", 
         contrast_limits=[0,255], 
         blending = 'additive', colormap = 'cyan')#
    viewer.add_labels(masks, name = f'trans{expt}/{pos}',
                   )
    viewer.show(block=True)

ND0012 Pos0
Using cropping: (1200, 1600)


In [26]:
trans = numpy.load(transform_path)

In [29]:
transform_path = f'/home/nathan/data/kraken/ras/{expt}/{pos}/gfp_transform_tensor_14.npy'


In [41]:
trans_13[115:125, :2, -1]

array([[-7.64561612e+01, -4.70585977e+02],
       [-7.57575709e+01, -4.70382052e+02],
       [-7.46870318e+01, -4.70267907e+02],
       [-7.77721680e+01, -4.70393597e+02],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 1.45408388e+00, -4.30155860e+00],
       [-4.98507831e-01, -2.32258565e+00],
       [ 2.20540161e+00, -3.94122041e+00],
       [-8.59118685e-02, -3.27027988e+00],
       [-7.60416375e-01, -3.32813527e+00]])

In [42]:
trans_14[115:125, :2, -1]

array([[-7.64561612e+01, -4.70585977e+02],
       [-7.57575709e+01, -4.70382052e+02],
       [-7.46870318e+01, -4.70267907e+02],
       [-7.77721680e+01, -4.70393597e+02],
       [ 0.00000000e+00,  0.00000000e+00],
       [ 1.45408388e+00, -4.30155860e+00],
       [-4.98507831e-01, -2.32258565e+00],
       [ 2.20540161e+00, -3.94122041e+00],
       [-8.59118685e-02, -3.27027988e+00],
       [-7.60416375e-01, -3.32813527e+00]])

In [49]:
sum(trans_14[119,:2, -1])

0.0

In [47]:
if trans_14[119,:2, -1] == [0,0]:
    print(True)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [69]:
i_list = []
for i in range(len(trans)):
    if sum(trans[i,:2, -1]) == 0:
        print(i)
        i_list.append(i)
edit = np.delete(trans, i_list, 0)

0
119
238
357
476
595
714
833
952
1071


In [71]:
len(edit)

1180

In [72]:
np.save(f'/home/nathan/data/kraken/ras/{expt}/{pos}/gfp_transform_tensor_16.npy', edit)

In [57]:
len(trans)

1190

In [59]:
import numpy as np

In [65]:
np.s_[119::]

slice(119, None, None)

In [68]:
len(np.delete(trans, [119], 0))

1189

# Load images

In [ ]:
# define experiment ID and select a position
expt = 'ND0010'
pos = 'Pos0'
# point to where the data is
root_dir = '/home/nathan/data/kraken/ras'
image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
transform_path = f'{root_dir}/{expt}/{pos}/gfp_transform_tensor.npy'
tracks_path = f'{root_dir}/{expt}/{pos}/tracks.h5'
# lazily load images
images = DaskOctopusLiteLoader(image_path, 
                               transforms = transform_path,
                               crop = (1200,1600),
                               remove_background = False)
images.channels

In [ ]:
gfp = images['gfp']
rfp = images['rfp']
irfp = images['irfp']
masks = images['mask']

In [ ]:
import numpy as np
for i in rfp[1109:1113]:
    print(np.mean(i.compute()))

### Add tracks

In [ ]:
import btrack
from btrack.utils import tracks_to_napari

#### iRFP tracks

In [ ]:
with btrack.dataio.HDF5FileHandler(tracks_path, 
                                   'r', obj_type='obj_type_1'
                                  ) as h:
    irfp_tracks = h.tracks

In [ ]:
irfp_tracks, irfp_properties, irfp_graph = tracks_to_napari(irfp_tracks, ndim = 2)

#### RFP tracks

In [ ]:
with btrack.dataio.HDF5FileHandler(tracks_path, 
                                   'r', obj_type='obj_type_2'
                                  ) as h:
    rfp_tracks = h.tracks

In [ ]:
rfp_tracks, rfp_properties, rfp_graph = tracks_to_napari(rfp_tracks, ndim = 2)

# Launch viewer

In [ ]:
viewer = napari.Viewer()
viewer.add_image(gfp, name="gfp", 
                 contrast_limits=[0,255],
                 blending = 'additive', colormap = 'green')#, is_pyramid=False)
viewer.add_image(rfp, name="rfp", 
                 contrast_limits=[0,55], 
                 blending = 'additive', colormap = 'magenta')#, is_pyramid=False)
viewer.add_image(irfp, name="irfp", 
                 contrast_limits=[0,255], 
                 blending = 'additive', colormap = 'cyan')#
viewer.add_image(masks, name = 'masks',
                contrast_limits=[0,255],
               blending = 'additive',)
viewer.add_tracks(irfp_tracks)
viewer.add_tracks(rfp_tracks)